### SQLite

In [4]:
import sqlite3
from sqlite3 import Error

In [ ]:
# Criar a conexão com o banco de dados, se não existir o arquivo será criado
conn = sqlite3.connect('aula04.db')

In [ ]:
# definindo um iterador que permitirá navegar e manipular os registros
cur = conn.cursor()

Criando tabelas no banco de dados

*books*: Tabela que armazenará livros, contará com as seguintes colunas:

- id: chave primária única que identificará um livro
- title: título do livro
- author_id: id do autor do livro (chave estrangeira)

*authors*: Tabela que armazenerá autores, contará com as seguintes colunas

- id: chave primária única que identificará o autor
- name: nome do autor
- born_date: data de nascimento do autor

In [5]:
sql_create_authors_table = """
CREATE TABLE IF NOT EXISTS authors(
	id integer PRIMARY KEY,
	name text NOT NULL,
	born_date text
);
"""

sql_create_books_table = """
CREATE TABLE IF NOT EXISTS books(
	id integer PRIMARY KEY,
	title text NOT NULL,
	author_id integer NOT NULL,
	FOREIGN KEY (author_id) REFERENCES authors (id)
);
"""

In [6]:
def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [7]:
create_table(conn, sql_create_authors_table)
create_table(conn, sql_create_books_table)

Criando registros no banco de dados

In [12]:
def create_records(conn, table, values):
	if table == "authors":
		sql = "INSERT INTO authors(name,born_date) VALUES(?,?)"
	elif table == "books":
		sql = "INSERT INTO books(title,author_id) VALUES(?,?)"
	if len(values)==2:
		with conn:cur.execute(sql, values)
	else:
		with conn:cur.executemany(sql, values)
	return cur.lastrowid

In [13]:
with conn:
	author_values = ('Aldous Huxley', '1894-07-26')
	author_id = create_records(conn, "authors", author_values)

	books_values = [
		('Brave New World', author_id),
		('The Perennial Philosophy', author_id),
		('The Doors of Perception', author_id),
		('The Art of Seeing', author_id),
		('Update Field', 5)
	]

	print(books_values)
	create_records(conn, "books", books_values)

[('Brave New World', 2), ('The Perennial Philosophy', 2), ('The Doors of Perception', 2), ('The Art of Seeing', 2), ('Update Field', 5)]


In [32]:
cur.execute('SELECT * from books')
result = cur.fetchall()
print(result)

[(1, 'Aldous Huxley', '1894-07-26'), (2, 'Aldous Huxley', '1894-07-26')]


In [15]:
for row in result:
    print(row)

(1, 'Brave New World', 2)
(2, 'The Perennial Philosophy', 2)
(3, 'The Doors of Perception', 2)
(4, 'The Art of Seeing', 2)
(5, 'Update Field', 5)


### SQLAlchemy

In [16]:
from sqlalchemy import create_engine, MetaData, Table, select, text

In [ ]:
#engine = create_engine("sqlite:///aula04.db", echo=True)
engine = create_engine("sqlite:///aula04.db")

In [21]:
sql = text('SELECT * from books')

In [22]:
with engine.connect() as conn:
    result = conn.execute(sql)
    print(result.fetchall())

[(1, 'Brave New World', 2), (2, 'The Perennial Philosophy', 2), (3, 'The Doors of Perception', 2), (4, 'The Art of Seeing', 2), (5, 'Update Field', 5)]


In [23]:
metadata = MetaData()

In [24]:
t = Table("books", metadata, autoload_with=engine)

In [31]:
[column.key for column in t.columns]

['id', 'title', 'author_id']

In [29]:
sql = (select(t.columns.title,t.c.author_id).where(t.c.author_id == 2))
print(sql)

SELECT books.title, books.author_id 
FROM books 
WHERE books.author_id = :author_id_1


In [30]:
with engine.connect() as conn:
    result = conn.execute(sql)
    print(result.fetchall())

[('Brave New World', 2), ('The Perennial Philosophy', 2), ('The Doors of Perception', 2), ('The Art of Seeing', 2)]
